# COINTEGRACIÓN Y MODELOS DE CORRECCION DE ERROR

En el capítulo anterior se presentaron  las pruebas de raíz unitaria para el análisis de integración y en este capítulo se retoman esos elementos para desarrollar la metodología de cointegración, que establece “la combinación lineal entre dos o más variables debe cumplir con la condición de ser estacionaria”, esto es:  que la combinación debe tener media, varianza y covarianza constante. El procedimiento de Engle y Granger consiste en utilizar el análisis de integración  en la combinación de las variables, con el objetivo de probar si cumplen con la condición de ser estacionaria para establecer que son cointegradas. A la ecuación estática que se utiliza para probar cointegración se le conoce como la relación de equilibrio de largo plazo y para modelar la dinámica de corto plazo al equilibrio de largo plazo, Engle y Granger postulan que es necesario construir el Modelo de Corrección de Error (MCE). Para aplicar las metodologías  de cointegración,  este capítulo se estructura de la siguiente manera: 1) El análisis de cointegración de Engle-Granger con pruebas de raíz unitaria; 2)  Prueba de Phillips y Ouliaris para cointegración; 3) Modelo de Corrección de Error con Engle-Granger; y, 4)  Metodología de cointegración de Johansen-Juselius. 

## Metodología de Engle-Granger aplicada a la función Consumo

En este ejemplo se utilizan las librerías urca para análisis de integración y cointegración para series de tiempo escrita por Bernhard Pfaff y  Matthieu Stigler (2013) y car para el análisis de regresión aplicada de  Fox y  Weisberg (2011). 

In [1]:
# Activo las librerías
library(urca)
library(car)

Loading required package: carData


In [2]:
load("Consumo.RData")

In [3]:
summary(Consumo)

    Periodo       cp_mex           pib_mex        
 1993/01: 1   Min.   :4855294   Min.   : 7817381  
 1993/02: 1   1st Qu.:5799294   1st Qu.: 9553135  
 1993/03: 1   Median :6986111   Median :10694080  
 1993/04: 1   Mean   :7073960   Mean   :10885032  
 1994/01: 1   3rd Qu.:8218569   3rd Qu.:12281916  
 (Other):85   Max.   :9734100   Max.   :14307437  
 NA's   : 1   NA's   :1         NA's   :1         

In [4]:
# Asigno las variables de serie de tiempo a pib_mex y calculo
# derivada primera y segunda

lpib_mex <- log(Consumo$pib_mex)
lcp_mex <- log(Consumo$cp_mex)
dlpib_mex <- diff(lpib_mex)
dlcp_mex <- diff(lcp_mex)

# Se asigna la variable periodo que ordena en el tiempo
periodo <- Consumo$Periodo

In [5]:
# Grafico las variables de los logaritmos del 
# pib_mex(PIB de México) y el cp_mex(Consumo de México)
plot(periodo, lpib_mex, main="Consumo y PIB de Mexico", ylim=c(15, 17))
lines(lpib_mex, col="black")
lines(lcp_mex, col="red")

Como se puede observar en la gráfica siguiente, las series del consumo y del PIB de México siguen una tendencias con respecto al tiempo, con caídas muy parecidas en los periodos de crisis económicas de 1994-1995 y 2008-2009.  

In [6]:
# Construyo el gráfico con la derivada primera del consumo 
# y del PIB

plot(dlpib_mex, main="Derivadas en logaritmo de consumo y PIB de Mexico")
lines(dlpib_mex, col="black")
lines(dlcp_mex, col="red")

De la gráfica siguiente con las primeras diferencias de los logaritmos  del consumo (rojo) y del pib (negro con círculos), los resultados muestran que las dos transformaciones son series sin tendencia. Un aspecto adicional importante es que el consumo tiene mayor variabilidad que el pib. La primera característica esta relacionada con series estacionarias en media y complementado con el análisis de integración se puede concluir que las dos variables son i(1). Aunque la segunda característica es importante para el análisis económico, en términos de los supuestos de estacionariedad lo relevante es que las varianzas sean estacionarias con respecto al tiempo y tal supuesto parece cumplirse para las dos series de tiempo.

Para seguir con la metodología de Engle-Granger, en primer lugar se estima la función consumo con elasticidades constantes 

In [7]:
# Función de consumo para probar cointegración tipo Engle y
# Granger

mod_1 <- lm(lcp_mex ~ lpib_mex)
summary(mod_1)


Call:
lm(formula = lcp_mex ~ lpib_mex)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.062111 -0.013747  0.000094  0.016850  0.057125 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -3.57902    0.24675  -14.51   <2e-16 ***
lpib_mex     1.19408    0.01524   78.35   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.02409 on 88 degrees of freedom
  (1 observation deleted due to missingness)
Multiple R-squared:  0.9859,	Adjusted R-squared:  0.9857 
F-statistic:  6138 on 1 and 88 DF,  p-value: < 2.2e-16


Los resultados muestran que la constante y el parámetro del logaritmo del PIB son altamente significativos y la elasticidad ingreso del consumo es de 1.19.  

In [8]:
# Genero un objeto con los residuales de la ecuación
res_1 <- residuals.lm(mod_1)
summary(res_1)

      Min.    1st Qu.     Median       Mean    3rd Qu.       Max. 
-6.211e-02 -1.375e-02  9.414e-05  0.000e+00  1.685e-02  5.713e-02 

In [9]:
residualPlot(mod_1)

La gráfica de los residuales muestra un comportamiento de tendencia con pendiente muy similar al cero o  sin un comportamiento definido, lo cual es importante para el cumplimiento de la estacionariedad o que sea I(0). 

En segundo lugar, a los residuales de la función consumo se le aplica las pruebas de raíz unitaria. 

In [10]:
# Prueba ADF para cointegración tipo Granger
lc.df <- ur.df(y=res_1,type="trend", lags=4, selectlags=c("AIC"))
summary(lc.df)


############################################### 
# Augmented Dickey-Fuller Test Unit Root Test # 
############################################### 

Test regression trend 


Call:
lm(formula = z.diff ~ z.lag.1 + 1 + tt + z.diff.lag)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0302503 -0.0060666 -0.0004746  0.0071103  0.0283283 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -3.545e-03  2.568e-03  -1.380  0.17140    
z.lag.1     -1.705e-01  6.408e-02  -2.661  0.00946 ** 
tt           5.919e-05  4.791e-05   1.236  0.22035    
z.diff.lag1 -2.209e-01  9.695e-02  -2.279  0.02540 *  
z.diff.lag2 -2.636e-01  9.231e-02  -2.855  0.00551 ** 
z.diff.lag3 -2.791e-01  8.946e-02  -3.119  0.00254 ** 
z.diff.lag4  5.527e-01  8.772e-02   6.301 1.63e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.01048 on 78 degrees of freedom
Multiple R-squared:  0.8037,	Adjusted R-squared:  0.7886 
F-st

La primera prueba ADF considera la tendencia y constante, y los resultados muestran que el valor del estadístico (-2.66) es menor en valor absoluto a valor critico al 10 porciento (3.15), por lo que los residuales tienen raíz unitaria por lo que se concluye que el consumo y el pib no mantienen una relación de cointegración. 

In [11]:
# La segunda prueba ADF se aplica solamente con constante

lc.df <- ur.df(y=res_1, type="drift", lags=4, selectlags=c("AIC"))
summary(lc.df)


############################################### 
# Augmented Dickey-Fuller Test Unit Root Test # 
############################################### 

Test regression drift 


Call:
lm(formula = z.diff ~ z.lag.1 + 1 + z.diff.lag)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.032166 -0.005711  0.000099  0.007042  0.026794 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.0007076  0.0011532  -0.614  0.54123    
z.lag.1     -0.1597353  0.0636988  -2.508  0.01420 *  
z.diff.lag1 -0.2053951  0.0964468  -2.130  0.03632 *  
z.diff.lag2 -0.2464100  0.0915675  -2.691  0.00869 ** 
z.diff.lag3 -0.2590241  0.0882744  -2.934  0.00437 ** 
z.diff.lag4  0.5730218  0.0864515   6.628  3.8e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.01051 on 79 degrees of freedom
Multiple R-squared:  0.7998,	Adjusted R-squared:  0.7871 
F-statistic: 63.13 on 5 and 79 DF,  p-value: < 2.2e-16


Value of test-sta

En este caso se encontró que aunque la diferencia es pequeña, el valor del estadístico (2.51) es menor en valor absoluto a valor critico al 10 porciento (-2.58),  por lo que la conclusión  es la misma que en al prueba anterior, el  consumo y el pib no están  cointegrados.

In [12]:
# Por último se aplica la prueba ADF sin tendencia y constante
lc.df <- ur.df(y=res_1, type='none', lags=4, selectlags=c("AIC"))
summary(lc.df)


############################################### 
# Augmented Dickey-Fuller Test Unit Root Test # 
############################################### 

Test regression none 


Call:
lm(formula = z.diff ~ z.lag.1 - 1 + z.diff.lag)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.032840 -0.006482 -0.000614  0.006472  0.026333 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
z.lag.1     -0.15752    0.06335  -2.487  0.01498 *  
z.diff.lag1 -0.19996    0.09566  -2.090  0.03977 *  
z.diff.lag2 -0.24062    0.09072  -2.652  0.00964 ** 
z.diff.lag3 -0.25287    0.08736  -2.895  0.00489 ** 
z.diff.lag4  0.57956    0.08546   6.782 1.86e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.01047 on 80 degrees of freedom
Multiple R-squared:  0.7989,	Adjusted R-squared:  0.7863 
F-statistic: 63.56 on 5 and 80 DF,  p-value: < 2.2e-16


Value of test-statistic is: -2.4866 

Critical values for test statistics: 
     1pct

Con esta especificación de la prueba, el valor del estadístico (-2.48) es mayor en valor absoluto a valor critico al 10 porciento (-1.61),  por lo que ahora la conclusión se modifica y se puede asegurar que el  consumo y el pib están  cointegrados.

In [13]:
# Análisis de cointegración de Phillips-Ouliaris para función
# de consumo
# Con el comando cbind integro las dos variables lpib_mex y 
# lcp_mex, en un solo objeto

ecb.consumo <- cbind(lcp_mex, lpib_mex)
ecb.consumo

lcp_mex,lpib_mex
15.42947,15.90066
15.44776,15.90177
15.45252,15.90261
15.51186,15.94013
15.45964,15.93199
15.50110,15.95618
15.50295,15.95007
15.55834,15.99131
15.40698,15.92094
15.39558,15.87186


In [14]:
# Aplico la prueba PO del tipo PZ

lc.po <- ca.po(ecb.consumo, type="Pz")

In [15]:
summary(lc.po)


######################################## 
# Phillips and Ouliaris Unit Root Test # 
######################################## 

Test of type Pz 
detrending of series none 

Response lcp_mex :

Call:
lm(formula = lcp_mex ~ zr - 1)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.158515 -0.029906  0.008515  0.030204  0.061358 

Coefficients:
           Estimate Std. Error t value Pr(>|t|)    
zrlcp_mex    0.7466     0.1063   7.026 4.49e-10 ***
zrlpib_mex   0.2470     0.1034   2.389   0.0191 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.04392 on 87 degrees of freedom
Multiple R-squared:      1,	Adjusted R-squared:      1 
F-statistic: 5.727e+06 on 2 and 87 DF,  p-value: < 2.2e-16


Response lpib_mex :

Call:
lm(formula = lpib_mex ~ zr - 1)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.077564 -0.015059  0.000777  0.020031  0.044753 

Coefficients:
           Estimate Std. Error t value Pr(>|t|)    
zr

Los resultados muestran que el estadístico calculado (10.99) es menor que el valor critico al 10 porciento (33.92) y todos los niveles de significancia, con lo que se concluye que se acepta la hipótesis nula de no cointegración.

In [16]:
lc.po <- ca.po(ecb.consumo, type="Pu")

In [17]:
summary(lc.po)


######################################## 
# Phillips and Ouliaris Unit Root Test # 
######################################## 

Test of type Pu 
detrending of series none 


Call:
lm(formula = z[, 1] ~ z[, -1] - 1)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.091371 -0.035614  0.007748  0.039446  0.063685 

Coefficients:
         Estimate Std. Error t value Pr(>|t|)    
z[, -1] 0.9730269  0.0002872    3388   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.04411 on 89 degrees of freedom
Multiple R-squared:      1,	Adjusted R-squared:      1 
F-statistic: 1.148e+07 on 1 and 89 DF,  p-value: < 2.2e-16


Value of test-statistic is: 9.3937 

Critical values of Pu are:
                  10pct    5pct    1pct
critical values 20.3933 25.9711 38.3413


La conclusión con la prueba Pz se confirma con el estadístico Pu; el estadístico calculado (9.39) es menor que el valor critico al 10 porciento (20.39) y todos los niveles de significancia, con lo que se concluye que se acepta la hipótesis nula de no cointegración.

In [18]:
# Modelo de correción de error
# Engle y Granger demuestran que si existe una doble inferencia.
# 1) Si un conjunto de variables están cointegradas también
# existirá un mecanismo de corrección, de error MCE para
# representar el proceso generador de los datos. y 
# 2) Si el PGD de las variables tiene una representación de MCE
# entonces esas variables estarán cointegradas. A esta inferencia
# se le conoce como el Teorema de Representación de Granger.

# Para poder estimar el modelo de corrección de error es muy
# importante tener asignadas la variables en logaritmo y primera
# diferencia del logaritmo, estimar el modelo de largo plazo,
# guardar los residuales y ahora el rezago de los residuos.

# Modelo de correción de Error: Metodología de Granger
mod_1 <- lm(lcp_mex ~ lpib_mex)
mod_1


Call:
lm(formula = lcp_mex ~ lpib_mex)

Coefficients:
(Intercept)     lpib_mex  
     -3.579        1.194  


In [19]:
summary(mod_1)


Call:
lm(formula = lcp_mex ~ lpib_mex)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.062111 -0.013747  0.000094  0.016850  0.057125 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -3.57902    0.24675  -14.51   <2e-16 ***
lpib_mex     1.19408    0.01524   78.35   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.02409 on 88 degrees of freedom
  (1 observation deleted due to missingness)
Multiple R-squared:  0.9859,	Adjusted R-squared:  0.9857 
F-statistic:  6138 on 1 and 88 DF,  p-value: < 2.2e-16


In [20]:
# Asigno los residuales del modelo estático (mod_1)
res_1 <- residuals.lm(mod_1)

In [21]:
# Se aplica un rezado a los residuales y asigno a  res_1.l

res_1.l = lag(res_1)

Entonces se estima la función de las diferencias del logaritmo del consumo (dlcp_mex) con respecto a las diferencias del logaritmo del pib (dlpib_mex) y de los residuales rezagados un periodo.  

In [22]:
mod_ce_1 <- lm(dlcp_mex ~ dlpib_mex + res_1.l)
summary(mod_ce_1)


Call:
lm(formula = dlcp_mex ~ dlpib_mex + res_1.l)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.054691 -0.011976  0.000953  0.014435  0.049647 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.001721   0.002101  -0.819    0.415    
dlpib_mex    1.423884   0.076888  18.519  < 2e-16 ***
res_1.l     -0.424926   0.086094  -4.936 3.87e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.01931 on 86 degrees of freedom
  (1 observation deleted due to missingness)
Multiple R-squared:  0.8206,	Adjusted R-squared:  0.8164 
F-statistic: 196.7 on 2 and 86 DF,  p-value: < 2.2e-16


Se observa el parámetro del vector de cointegración. Es negativo y en términos absolutos menos que uno (-0.43), por lo que se acepta que la relación de corto plazo tiende a la de largo plazo. La elasticidad ingreso del consumo de corto plazo es 1.42 y de largo plazo es 1.19, lo que indica una sobrereacción del consumo en el corto plazo.

#### COINTEGRACIÓN CON METODOLOGÍA DE JOHANSEN Y JOSELIUS 

Con este método es posible probar tanto el orden de integración de un conjunto de variables,  como  la existencia de cointegración entre las mismas. 

In [24]:
# Aplico la prueba de traza de cointegración sin tendencia y constante
summary(ca.jo(ecb.consumo, type="trace", ecdet="none", spec=c("longrun"), K=4))


###################### 
# Johansen-Procedure # 
###################### 

Test type: trace statistic , with linear trend 

Eigenvalues (lambda):
[1] 0.1751774 0.0169835

Values of teststatistic and critical values of test:

          test 10pct  5pct  1pct
r <= 1 |  1.47  6.50  8.18 11.65
r = 0  | 18.04 15.66 17.95 23.52

Eigenvectors, normalised to first column:
(These are the cointegration relations)

            lcp_mex.l4 lpib_mex.l4
lcp_mex.l4    1.000000    1.000000
lpib_mex.l4  -1.280708   -1.037871

Weights W:
(This is the loading matrix)

           lcp_mex.l4 lpib_mex.l4
lcp_mex.d  -0.3578910 -0.08189816
lpib_mex.d -0.1079621 -0.07962291


Los resultados muestran que para la primera hipótesis r =0 y alternativa r=1, se acepta la hipótesis alternativa de cointegración: el estadístico es mayor al valor critico del 10 porciento,  18.04 > 15.56. Para la segunda hipótesis r<=1 y alternativa r=2, se acepta la hipótesis nula de un solo vector de cointegración. Por lo que se puede concluir que existe un vector de cointegración y se representa por [1 -1.28].

In [25]:
# Prueba de Johansen de la traza con constante

summary(ca.jo(ecb.consumo, type="trace", ecdet="const", spec=c('longrun'), K=4))


###################### 
# Johansen-Procedure # 
###################### 

Test type: trace statistic , without linear trend and constant in cointegration 

Eigenvalues (lambda):
[1] 2.017097e-01 1.265562e-01 4.385381e-15

Values of teststatistic and critical values of test:

          test 10pct  5pct  1pct
r <= 1 | 11.64  7.52  9.24 12.97
r = 0  | 31.01 17.85 19.96 24.60

Eigenvectors, normalised to first column:
(These are the cointegration relations)

            lcp_mex.l4 lpib_mex.l4  constant
lcp_mex.l4    1.000000    1.000000  1.000000
lpib_mex.l4  -1.255645   -1.414691 -1.064081
constant      4.557479    7.198639  1.483342

Weights W:
(This is the loading matrix)

           lcp_mex.l4 lpib_mex.l4      constant
lcp_mex.d  -0.4955292  0.05574003 -7.778578e-12
lpib_mex.d -0.2795953  0.09201028 -6.742190e-12


Los resultados muestran que para la primera hipótesis r =0 y alternativa r=1, se acepta la hipótesis alternativa de cointegración: el estadístico es mayor al valor critico del 10 porciento,  31.01 > 17.85. Para la segunda hipótesis r<=1 y alternativa r=2, se acepta la hipótesis alternativa nula de dos vector de cointegración. Por lo que se pueden identificar  los vectores de cointegración [1 -1.25 4.55] y [1 -1.41 7.19].

In [26]:
# Prueba de Johansen de la traza con tendencia   
summary(ca.jo(ecb.consumo, type="trace", ecdet="trend", spec=c("longrun"), K=4))


###################### 
# Johansen-Procedure # 
###################### 

Test type: trace statistic , with linear trend in cointegration 

Eigenvalues (lambda):
[1] 3.007210e-01 6.711864e-02 9.635847e-18

Values of teststatistic and critical values of test:

          test 10pct  5pct  1pct
r <= 1 |  5.98 10.49 12.25 16.26
r = 0  | 36.74 22.76 25.32 30.45

Eigenvectors, normalised to first column:
(These are the cointegration relations)

              lcp_mex.l4  lpib_mex.l4     trend.l4
lcp_mex.l4   1.000000000  1.000000000  1.000000000
lpib_mex.l4 -2.613552540 -0.740358526 -1.944486294
trend.l4     0.008212396 -0.002968024  0.007611815

Weights W:
(This is the loading matrix)

            lcp_mex.l4 lpib_mex.l4      trend.l4
lcp_mex.d  -0.10989875  -0.3329111 -5.057755e-12
lpib_mex.d  0.03206499  -0.2557477 -1.062532e-12


Al igual que en el caso de la opción sin tendencia y constante,  se acepta la hipótesis de cointegración con solo vector. El vector de cointegración es  [1  -2.61  0.008]. 